###  Librerías importadas

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import pickle
from tensorflow import keras
from datetime import datetime
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

import time

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

### Hiperparámetros

In [ ]:
csv_path          = "regina.csv"
dropped_features  = ['Hi Temp', 'Low Temp', 'Wind Chill', 'Heat Index', 'THSW Index', 'THW Index', 'Wind Run', 'Solar Energy', 'Hi Solar Rad.', 'In Heat', 'ISS Recept', 'Arc. Int']

train_perc = .8
val_perc   = .1

sequence_length = 90
offset          = 0
sampling_rate   = 1
length          = 6 #horas
min_temp        = 0.5
batch_size      = 256

learning_rate   = 0.001
epochs          = 100

### Read CSV

In [ ]:
dateparse = lambda x: datetime.strptime(x, '%d.%m.%y %H:%M')

wind_dic = {'---' : 0,
            'E': 1,
            'W': 2,
            'N': 3,
            'S': 4,
            'NE': 5,
            'SE': 6,
            'NW': 7,
            'SW': 8,
            'ENE': 9,
            'NNE': 10,
            'WNW': 11,
            'NNW': 12,
            'ESE': 13,
            'SSE': 14,
            'WSW': 15,
            'SSW': 16}

data = pd.read_csv(csv_path, parse_dates=['Date Time'], date_parser=dateparse, na_values=['---', '------'], converters={'Wind Dir': lambda x: wind_dic[x], 'Hi Dir': lambda x: wind_dic[x]}, index_col=0)

### Agregamos los datos que faltan

In [ ]:
data['Temp Out'] = data['Temp Out'].apply(lambda x: x-2)
data['Temp Out'] = data['Temp Out'].apply(lambda x: np.nan if x <= -10 else x)

data = data.resample('10min', origin='start').mean()

data['Wind Dir'] = data['Wind Dir'].replace(np.nan, 0)
data['Hi Dir'] = data['Hi Dir'].replace(np.nan, 0)

data = data.interpolate()

data.info()

### Convertimos estampillas de tiempo

In [ ]:
timestamp_s = data.index
timestamp_s = timestamp_s.map(pd.Timestamp.timestamp)

day = 24*60*60
year = 365.2425 * day

data['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
data['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
data['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
data['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

### Correlación de los datos

In [ ]:
data.corr()

### Dropeamos las características que no tienen correlación

In [ ]:
df = data.drop(dropped_features, axis = 1)

### Normalización de los datos

In [ ]:
df_length = len(df)

train_length = int(df_length*train_perc)
val_length   = int(df_length*(train_perc+val_perc))

train_mean = df.values[:train_length].mean(axis=0)
train_std  = df.values[:train_length].std(axis=0)

df_norm = (df.values - train_mean) / train_std
df_norm = pd.DataFrame(df_norm)

In [ ]:
file = open("norm.params", "wb")
pickle.dump([train_mean, train_std, wind_dic], file)
file.close()

### Dividisión de los datos

In [ ]:
train_df = df_norm[:train_length-1]
val_df   = df_norm[train_length:val_length-1]
test_df  = df_norm[val_length:]

###  Creación de secuencias

In [ ]:
def create_y(df, start, end, length, min_temp):
    y_data = np.arange(end-start)
    
    df_bool = df['Temp Out'][start:end+length] <= min_temp
        
    for i in range(0, end-start):
        y_data[i] = 1 if np.any(df_bool[i:i+length]) else 0

    return y_data

In [ ]:
start = sequence_length*sampling_rate + offset*sampling_rate
end   = train_length + offset*sampling_rate

x_train = train_df
y_train = create_y(df, start, end, length*6, min_temp)

dataset_train = keras.preprocessing.timeseries_dataset_from_array(
    x_train.values,
    y_train,
    sequence_length = sequence_length,
    sampling_rate = sampling_rate,
    shuffle=True,
    batch_size = batch_size
)

In [ ]:
start = train_length + sequence_length*sampling_rate + offset*sampling_rate
end   = val_length + offset*sampling_rate

x_val = val_df
y_val = create_y(df, start, end, length*6, min_temp)

dataset_val = keras.preprocessing.timeseries_dataset_from_array(
    x_val.values,
    y_val,
    sequence_length = sequence_length,
    sampling_rate = sampling_rate,
    shuffle=False,
    batch_size = batch_size
)

In [ ]:
start = val_length + sequence_length*sampling_rate + offset*sampling_rate
x_end = len(test_df) - sequence_length*sampling_rate - offset*sampling_rate

x_test = test_df[:x_end]
y_test = create_y(df, start, df_length, length*6, min_temp)

dataset_test = keras.preprocessing.timeseries_dataset_from_array(
    x_test.values,
    y_test,
    sequence_length = sequence_length,
    sequence_stride = sampling_rate,
    sampling_rate = sampling_rate,
    shuffle=False,
    batch_size = batch_size
)

### Creación del modelo

In [ ]:
for batch in dataset_train.take(1):
    x, y = batch

input_shape = x.shape[1], x.shape[2]

In [ ]:
inputs = keras.layers.Input(input_shape)
lstm_layer = keras.layers.LSTM(4, dropout=0.2, recurrent_dropout=0.2, return_sequences=False)(inputs)
output = keras.layers.Dense(1, activation="sigmoid")(lstm_layer)

model = keras.Model(inputs, output)
model.compile(keras.optimizers.Adam(learning_rate), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False))
model.summary()

### Entrenamiento del modelo

In [ ]:
path_checkpoint = "modelo_pronosticador_regina.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=5)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_best_only=True,
)

history = model.fit(dataset_train, epochs = epochs, validation_data=dataset_val, callbacks = [es_callback, modelckpt_callback])

In [ ]:
plt.rcParams['figure.figsize'] = [6, 4]

def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.ylim(0.05, 0.18)
    plt.legend()
    plt.show()

visualize_loss(history, "Training and Validation Loss")

### Mostramos los resultados

In [ ]:
i = start

test_real_y = np.zeros(0)
test_pred_y = np.zeros(0)
fechas  = np.zeros(0, dtype='datetime64')

plt.rcParams['figure.figsize'] = [15, 5]

test_size  = tf.data.experimental.cardinality(dataset_test).numpy()

for batch in dataset_test.take(test_size):
    x, y = batch
    y_pred = model.predict(x)
    
    b_size = x.shape[0]
    
    test_real_y  = np.concatenate((test_real_y, y))
    test_pred_y  = np.concatenate((test_pred_y, np.squeeze(y_pred)))
    fechas  = np.concatenate((fechas, np.squeeze(np.array(data.index)[i:i+b_size])))
    
    fig, ax = plt.subplots()
    
    ax.plot(data.index[i:i+b_size], y[0:b_size], c='r', label="Test Data")
    ax.plot(data.index[i:i+b_size], (y_pred[0:b_size]),c='g', label="Prediction")
    
    i += b_size
    
    plt.ylim(0, 1.1)
    plt.legend()
    plt.show()

### Métricas

In [ ]:
train_size = tf.data.experimental.cardinality(dataset_train).numpy()
val_size   = tf.data.experimental.cardinality(dataset_val).numpy()
test_size = (test_size-1)*batch_size + b_size

In [ ]:
train_real_y = np.zeros(0)
train_pred_y = np.zeros(0)

for batch in dataset_train.take(train_size):
    x, y = batch
    y_pred = model.predict(x)

    last_batch_size = x.shape[0]
    
    train_real_y  = np.concatenate((train_real_y, y))
    train_pred_y  = np.concatenate((train_pred_y, np.squeeze(y_pred)))
    
train_size = (train_size-1)*batch_size + last_batch_size

In [ ]:
val_real_y = np.zeros(0)
val_pred_y = np.zeros(0)

for batch in dataset_val.take(val_size):
    x, y = batch
    y_pred = model.predict(x)
    
    last_batch_size = x.shape[0]

    val_real_y  = np.concatenate((val_real_y, y))
    val_pred_y  = np.concatenate((val_pred_y, np.squeeze(y_pred)))

val_size = (val_size-1)*batch_size + last_batch_size

In [ ]:
conf_matrix_train = confusion_matrix(train_real_y, np.around(train_pred_y))
conf_matrix_val   = confusion_matrix(val_real_y, np.around(val_pred_y))
conf_matrix_test  = confusion_matrix(test_real_y, np.around(test_pred_y)) 

ConfusionMatrixDisplay(conf_matrix_test, display_labels=['No Heló', 'Heló']).plot()

plt.show()

In [ ]:
tn_train, fp_train, fn_train, tp_train = conf_matrix_train.ravel()
tn_val, fp_val, fn_val, tp_val         = conf_matrix_val.ravel()
tn_test, fp_test, fn_test, tp_test     = conf_matrix_test.ravel()

def accuracy(tn, fp, fn, tp): return (tp+tn)/(tp+tn+fp+fn)
def precision(tp, fp): return tp/(tp+fp)
def recall(tp, fn): return tp/(tp+fn)
def f1(tp, fp, fn): return (2*tp)/(2*tp+fp+fn)

stats = pd.DataFrame({'Dataset':   ['train', 'val', 'test'],
                      'Size':      [train_size, val_size, test_size],
                      'Accuracy':  [accuracy(tn_train, fp_train, fn_train, tp_train), accuracy(tn_val, fp_val, fn_val, tp_val), accuracy(tn_test, fp_test, fn_test, tp_test)],
                      'Precision': [precision(tp_train, fp_train), precision(tp_val, fp_val), precision(tp_test, fp_test)],
                      'Recall':    [recall(tp_train, fn_train), recall(tp_val, fn_val), recall(tp_test, fn_test)],
                      'F1':        [f1(tp_train, fp_train, fn_train), f1(tp_val, fp_val, fn_val), f1(tp_test, fp_test, fn_test)]})

stats.style.hide_index()

In [ ]:
fechas = pd.to_datetime(fechas)

indexes = np.squeeze(np.argwhere((fechas.hour >= 20) & (fechas.hour <= 23)))

noches_real  = test_real_y[indexes]
noches_pred  = test_pred_y[indexes]
fechas_noche = fechas[indexes]

indexes = np.squeeze(np.argwhere((fechas_noche.hour == 23) & (fechas_noche.minute == 50)))

noches_real_split = np.split(noches_real, indexes)
noches_pred_split = np.split(noches_pred, indexes)

print('METRICAS')
print('Estadísticas teniendo en cuenta el período de 20hs a 00hs')
print()
print('Dataset Test:')
print('Cantidad de noches predecidas / Cantidad de noches que heló: ', sum((np.any(noches_real_split[i] == 1) & (np.any(pred >= 0.5))) for i, pred in enumerate(noches_pred_split)), " / ", sum(np.any(x == 1) for x in noches_real_split))
print('Cantidad de noches con falsos positivos / Cantidad de noches que no heló: ', sum((np.all(noches_real_split[i] == 0) & (np.any(pred >= 0.5))) for i, pred in enumerate(noches_pred_split)), " / ", sum(np.all(x == 0) for x in noches_real_split))
print()

noches_criticas_real = noches_real[(fechas_noche.month == 8) | (fechas_noche.month == 9) | (fechas_noche.month == 10)]
noches_criticas_pred = noches_pred[(fechas_noche.month == 8) | (fechas_noche.month == 9) | (fechas_noche.month == 10)]
fechas_criticas      = fechas_noche[(fechas_noche.month == 8) | (fechas_noche.month == 9) | (fechas_noche.month == 10)]

indexes = np.squeeze(np.argwhere((fechas_criticas.hour == 23) & (fechas_criticas.minute == 50)))

noches_criticas_real = np.split(noches_criticas_real, indexes)
noches_criticas_pred = np.split(noches_criticas_pred, indexes)

print('Dataset Test (meses 8, 9 y 10):')
print('Cantidad de noches predecidas / Cantidad de noches que heló: ', sum((np.any(noches_criticas_real[i] == 1) & (np.any(pred >= 0.5))) for i, pred in enumerate(noches_criticas_pred)), " / ", sum(np.any(x == 1) for x in noches_criticas_real))
print('Cantidad de noches con falsos positivos / Cantidad de noches que no heló: ', sum((np.all(noches_criticas_real[i] == 0) & (np.any(pred >= 0.5))) for i, pred in enumerate(noches_criticas_pred)), " / ", sum(np.all(x == 0) for x in noches_criticas_real))